In [1]:
#@title Install Deps
#!pip install -q keras-cv
!pip install -q keras-cv
!pip install -q segmentation-models
!pip install -q -U albumentations
!pip install -U -q wandb
!pip install -q gdown
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [2]:
#@title wandb
import wandb
from wandb.keras import WandbCallback, WandbModelCheckpoint

wandb.login()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
#@title Imports
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras import backend as K
from pathlib import Path


from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, UpSampling2D, Dense, Multiply, SpatialDropout2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2DTranspose, Concatenate, Input, GlobalMaxPooling2D, Reshape
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras import layers
from tensorflow.keras.models import Model

from keras_cv.layers import SqueezeAndExcite2D
from keras_cv import layers as kcv_layers
import segmentation_models as sm

Using TensorFlow backend
Segmentation Models: using `tf.keras` framework.


In [4]:
import contextlib
@contextlib.contextmanager
def options(options):
  old_opts = tf.config.optimizer.get_experimental_options()
  tf.config.optimizer.set_experimental_options(options)
  try:
    yield
  finally:
    tf.config.optimizer.set_experimental_options(old_opts)

In [5]:
#@title Setup Config

config = dict(
    architecture = "ColonSegNet_cbam_heavy_aug_rd",
    seed = 94109,
    img_height = 256,
    img_width = 256,
    img_channels = 3,
    batch_size = 8,
    epochs = 200,
    learning_rate = 0.0003,
    weight_decay=0.0,
    cbam_ratio=16,
    dataset = "Kvasir-SEG",
    dataset_splits = {"train": 880, "val": 120},
    train_data_augmentation_config = dict(
                        rescale=1./255.,
                        rotation_range = 45,
                        brightness_range = [0.3, 1.3],
                        shear_range=0.3,
                        zoom_range=0.3,
                        channel_shift_range=0.4,
                        horizontal_flip=True,
                        vertical_flip=True,
                        cval=0.0,
                        fill_mode='reflect')
)

wandb.init(config=config, project="grad_proj", name=f"{config['architecture']}_LR{config['learning_rate']}")

wandb: Currently logged in as: abdelrhmannile (eva01). Use `wandb login --relogin` to force relogin


In [6]:
wandb.config

{'architecture': 'ColonSegNet_cbam_heavy_aug_rd', 'seed': 94109, 'img_height': 256, 'img_width': 256, 'img_channels': 3, 'batch_size': 8, 'epochs': 200, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'cbam_ratio': 16, 'dataset': 'Kvasir-SEG', 'dataset_splits': {'train': 880, 'val': 120}, 'train_data_augmentation_config': {'rescale': 0.00392156862745098, 'rotation_range': 45, 'brightness_range': [0.3, 1.3], 'shear_range': 0.3, 'zoom_range': 0.3, 'channel_shift_range': 0.4, 'horizontal_flip': True, 'vertical_flip': True, 'cval': 0.0, 'fill_mode': 'reflect'}}

In [7]:
#@title set seed for reproducibillity
random.seed(wandb.config["seed"])
np.random.seed(wandb.config["seed"])
tf.random.set_seed(wandb.config["seed"])
keras.utils.set_random_seed(wandb.config["seed"])

In [8]:
#@title download dataset
!wget https://datasets.simula.no/downloads/kvasir-seg.zip

--2023-11-02 13:37:32--  https://datasets.simula.no/downloads/kvasir-seg.zip
Resolving datasets.simula.no (datasets.simula.no)... 128.39.36.14
Connecting to datasets.simula.no (datasets.simula.no)|128.39.36.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46227172 (44M) [application/zip]
Saving to: ‘kvasir-seg.zip’

kvasir-seg.zip      100%[===================>]  44.08M  14.7MB/s    in 3.0s    

2023-11-02 13:37:36 (14.7 MB/s) - ‘kvasir-seg.zip’ saved [46227172/46227172]



In [9]:
#@title extract dataset
!unzip /kaggle/working/kvasir-seg.zip

Archive:  /kaggle/working/kvasir-seg.zip
   creating: Kvasir-SEG/
  inflating: Kvasir-SEG/kavsir_bboxes.json  
   creating: Kvasir-SEG/images/
  inflating: Kvasir-SEG/images/ck2bxiswtxuw80838qkisqjwz.jpg  
  inflating: Kvasir-SEG/images/ck2bxknhjvs1x0794iogrq49k.jpg  
  inflating: Kvasir-SEG/images/ck2bxlujamu330725szlc2jdu.jpg  
  inflating: Kvasir-SEG/images/ck2bxpfgxu2mk0748gsh7xelu.jpg  
  inflating: Kvasir-SEG/images/ck2bxqz3evvg20794iiyv5v2m.jpg  
  inflating: Kvasir-SEG/images/ck2bxskgxxzfv08386xkqtqdy.jpg  
  inflating: Kvasir-SEG/images/ck2bxw18mmz1k0725litqq2mc.jpg  
  inflating: Kvasir-SEG/images/ck2395w2mb4vu07480otsu6tw.jpg  
  inflating: Kvasir-SEG/images/ck2da7fwcjfis07218r1rvm95.jpg  
  inflating: Kvasir-SEG/images/cjyzjzssvd8pq0838f4nolj5l.jpg  
  inflating: Kvasir-SEG/images/cjyzk8qieoboa0848ogj51wwm.jpg  
  inflating: Kvasir-SEG/images/cju5hi52odyf90817prvcwg45.jpg  
  inflating: Kvasir-SEG/images/cju5hjxaae3i40850h5z2laf5.jpg  
  inflating: Kvasir-SEG/images/cju5hl8

In [10]:
train_split = """cju0qkwl35piu0993l0dewei2
cju0qoxqj9q6s0835b43399p4
cju0qx73cjw570799j4n5cjze
cju0roawvklrq0799vmjorwfv
cju0rx1idathl0835detmsp84
cju0s2a9ekvms080138tjjpxr
cju171py4qiha0835u8sl59ds
cju175facms5f0993a5tjikvt
cju17bz250pgd0799u1hqkj5u
cju17g6ykn1cs0993dww6qdi8
cju17hw9hr9c5098800fu4u8e
cju17otoe119u0799nqcbl8n1
cju17r8il13910799dr2wme2e
cju17v6ih0u7808783zcbg1jy
cju17x0j4nfc10993y31pvlgs
cju17z0qongpa0993de4boim4
cju1819curo000988pd5xcqme
cju183od81ff608017ekzif89
cju1871y11d6r0799k6cw4yze
cju18849rrsgr0988p90hkygb
cju18gzrq18zw0878wbf4ftw6
cju18ibp219ub08783i6o98g7
cju18kevfrojc0835bn90f1in
cju1alwgo30z60855fm3y23sm
cju1amqw6p8pw0993d9gc5crl
cju1aqqv02qwz0878a5cyhr67
cju1ats0y372e08011yazcsxm
cju1b0y2e396p08558ois175d
cju1b3zgj3d8e0801kpolea6c
cju1b75x63ddl0799sdp0i2j3
cju1bhnfitmge0835ynls0l6b
cju1bm8063nmh07996rsjjemq
cju1brhsj3rls0855a1vgdlen
cju1c0qb4tzi308355wtsnp0y
cju1c3218411b08014g9f6gig
cju1c4fcu40hl07992b8gj0c8
cju1c6yfz42md08550zgoz3pw
cju1c8ffau5770835g0g343o8
cju1cbokpuiw70988j4lq1fpi
cju1cdxvz48hw0801i0fjwcnk
cju1cfhyg48bb0799cl5pr2jh
cju1cj3f0qi5n0993ut8f49rj
cju1cnnziug1l0835yh4ropyg
cju1cqc7n4gpy0855jt246k68
cju1csmlc4ht10799b8ymmghg
cju1cu1u2474n0878tt7v4tdr
cju1cvkfwqrec0993wbp1jlzm
cju1cyjb5qtie0993njqne9m3
cju1d31sp4d4k0878r3fr02ul
cju1d50a94qf50855wsowacrc
cju1d96gsv62d09881b3wecw2
cju1ddr6p4k5z08780uuuzit2
cju1dfeupuzlw0835gnxip369
cju1dg44i4z3w0801nyz4p6zf
cju1dhfok4mhe0878jlgrag0h
cju1dia8hvc6v098827mgffnm
cju1djtprvd7b0988thwwrg09
cju1dnz61vfp40988e78bkjga
cju1dq3x1vgx109889c7wyirg
cju1drnhbrb9409935wi7vkhg
cju1efbr0rqxz09931z0lf4vf
cju1egh885m1l0855ci1lt37c
cju1egx9pvz2n0988eoy8jp23
cju1ejj7dvqfa0835ra184v5m
cju1erep75us208553i4ofwwe
cju1euant5l960878iqj5vvto
cju1euuc65wm00799m4sjdnnn
cju1ewnoh5z030855vpex9uzt
cju1expq45zst0855rjqwwj4m
cju1f15k3w4ct0835cmde6ypo
cju1f320ewfyu0988ndz6blh5
cju1f5x1164xv08555654c24r
cju1f79yhsb5w0993txub59ol
cju1f8w0t65en0799m9oacq0q
cju1fb9236a110801yvg0fwju
cju1ffnjn6ctm08015perkg37
cju1fj6axwfp30835ukhuzhw9
cju1fjsb4sipq09931lvd8e41
cju1fm3id6gl50801r3fok20c
cju1fmsyf6gxb0801cimx2gle
cju1fr4etsmrr09933u4t4aql
cju1ftaji6isw0855108yqcse
cju1fuoa4wmc50835qfd11sp9
cju1fyb1d69et0878muzdak9u
cju1g20bdwq6u0835e16xugcd
cju1g4nsb6ngy0799l4ezm8ab
cju1gghyjwxt80835vx0wgxw0
cju1gi7jlwyld0835cdf6g6qz
cju1gkndf6yi10801o1qnje19
cju1gv7106qd008784gk603mg
cju1h5w4wxajx0835mc954kxy
cju1h89h6xbnx08352k2790o9
cju1haab178i70799tk9z8y8x
cju1hhj6mxfp90835n3wofrap
cju1hirfi7ekp0855q0vgm9qq
cju1hmff8tkp809931jps6fbr
cju1hp9i2xu8e0988u2dazk7m
cju1hs0za7jha0855vj0mdrjt
cju1hyolc7aqu0878rrkfn1lr
cju2hdr06v2bq0799mbm3bks1
cju2hewssldzx0835ep795xu0
cju2hfqnmhisa0993gpleeldd
cju2hgsptlfam0835o3b59h1o
cju2hjrqcvi2j0801bx1i6gxg
cju2hlm19vjjf0801o69qnber
cju2hos57llxm08359g92p6jj
cju2hqt33lmra0988fr5ijv8j
cju2htabevq9108015qjei0x7
cju2hugv9vget0799hhk7ksvg
cju2hw5gjlr5h0988so2qqres
cju2hx006vidl0799igm81vmh
cju2i03ptvkiu0799xbbd4det
cju2i3hzclw3o0988rrgh911i
cju2i6acqvo6l0799u20fift8
cju2i8br1vqtd08784u6vmcjk
cju2iatlki5u309930zmgkv6h
cju2igw4gvxds0878808qj398
cju2ij9uiic2l09933ljiv6gm
cju2lberzkdzm09938cl40pog
cju2lcyfgkf5809932fn9gucn
cju2lejzcy4pc0878c9rlonot
cju2lyynuymli0855g7fxgbhe
cju2lz8vqktne0993fuym6drw
cju2m56cryvqd0801gtn2yp8t
cju2m71z2ywwv080131bcrsd3
cju2ma647l0nj0993ot4deq2q
cju2mfjndoz700988b9lc3zeq
cju2mh8t6p07008350e01tx2a
cju2nbdpmlmcj0993s1cht0dz
cju2nd7l7z98o0799gfjvyfmw
cju2nfnvxzdkd0878399axlco
cju2nguelpmlj0835rojdn097
cju2nnqrqzp580855z8mhzgd6
cju2np2k9zi3v079992ypxqkn
cju2nqapmzvk20801f9us40dx
cju2nsmwjlzyl0993jl80chvz
cju2ntxtdzlvu0799xl3j9pan
cju2nyc5f02m40801ojqbtiea
cju2oi8sq0i2y0801mektzvw8
cju2okvco06xc0799kxe5n1qh
cju2omjpeqj5a0988pjdlb8l1
cju2oo0wh0bqy0878biujeyhe
cju2oq5570avm079959o20op1
cju2osuru0ki00855txo0n3uu
cju2otvvv0l7z0855x7we8cb0
cju2ouil2mssu0993hvxsed6d
cju2p0eveqtdc0835gpi3p93i
cju2p4ddkmzxj0993p94o62av
cju2p91qir00k08350ddfif0w
cju2pag1f0s4r0878h52uq83s
cju2phaksnahz0993yxogjcpv
cju2pjb9v0ywn0878j5g5n69j
cju2pkwt3r8b90988v2ywq1px
cju2pmhtr17a00855cvpelzb0
cju2qdj95ru8g09886gfi9rsz
cju2qfie4rvz508357kad9z5o
cju2qh5le1ock0878oahaql7d
cju2qn2fzs1vy0988l243cvzy
cju2qozsk20cq0855ugrg3cri
cju2qqn5ys4uo0988ewrt2ip2
cju2qs32r1vys07999conmbvx
cju2qtee81yd708787bsjr75d
cju2qu37qobl50993aw7ghcfq
cju2qvuj1s9ok0835tp2k4ozh
cju2qxxko229x08786gvxxhur
cju2qz06823a40878ojcz9ccx
cju2r11x7sdgx0988o8ule0wl
cju2r2obh2bjm08553kng0rh7
cju2r6mt2om21099352pny5gw
cju2r7h21sj9608354gzks3ae
cju2r91dg2k090801bh0xzbxk
cju2raxlosl630988jdbfy9b0
cju2rga4psq9n09881z519xx0
cju2ricdv2iys0878sv1adh0u
cju2rkjfwoxys0993x768l1j2
cju2rlqdnoz9k0993cpjae3x0
cju2rmd2rsw9g09888hh1efu0
cju2rn0hasxri0835nfy3buay
cju2rnkt22xep0801as160g9t
cju2ro5jqsy680988pi6qsujw
cju2rpa30t07b0835im0erql0
cju2rqo702wpx0855fn7d5cxh
cju2rxm8rpbaf0993o3qr2oph
cju2rz4k434s70855wwx3ddtx
cju2rzpsmtb0f0835jabkbao1
cju2s16zp317h0799gr67jqc2
cju2s2527pfyr0993l3h1149a
cju2s9g11pnra0993gn4eh793
cju2saez63gxl08559ucjq3kt
cju2sevf53lkx08558h5bpaig
cju2sggy13na70855tbeoqgha
cju2spdagu1l50835da1f46fr
cju2srvy5440s0801y1ba9akr
cju2sszfq3uye0878sucelzk2
cju2suk42469908015ngmq6f2
cju2sxf3iqbpv09937iksn8ep
cju2syxa93yw40799x2iuwabz
cju2t16vuucaq0835xcpsivn2
cju2t2ivz43i10878zeg8r1br
cju2t3ibkuecr0835o7si16zv
cju2t62nq45jl0799odpufwx6
cju2t9tdwuk700835kv0ljmtl
cju2ti1du4idn0878giuozonw
cju2tjrog4jy30878pawyazqc
cju2top2ruxxy0988p1svx36g
cju2tpfa5uyx408359datxqqj
cju2tqfgw4oat0799rn0g5b2z
cju2trbpkv0c00988hxla5dzz
cju2trtjf4qjd0878a2zle9v9
cju2tvrvm53ws0801a0jfjdxg
cju2txjfzv60w098839dcimys
cju2tzypl4wss0799ow05oxb9
cju2u2b9o4zvp08788qb9nqxj
cju2u4pymvc720988wsxrmi84
cju2u73dj53oz0878486k8k4b
cju2ueb6j5ado0878vf5md13o
cju2ulk385h170799rlklxob0
cju2uokeg5jm20799xwgsyz89
cju2upu4evw7g08358guwozxv
cju2urqpwvxw70835rvndvtsi
cju2uwz9f5yf1085506cfamfx
cju2uy8ox62jo0801g88hh42z
cju2uzabhs6er0993x3aaf87p
cju2wtwj87kys0855kx6mddzw
cju2wve9v7esz0878mxsdcy04
cju2wx0gh7fpz0878wwyd9ep8
cju2wxv0hxs2f09884w48v8fi
cju2wzu8wxtgu09880ku9x1pg
cju2x7vw87mu30878hye2ca0m
cju2xa3i4y0160988i679zsqd
cju2xbk0080y80801eghyddi2
cju2xd75m82720801q4s4ik3n
cju2xf8e5y2wm08359vcgk09b
cju2xjz2ju8pe0993ysv9wg17
cju2xlcqxy9c60988vjacdznb
cju2xs6na81t20878pt6nkfip
cju2xwm1s84l50799i60mq0pu
cju2xyd9vyi7m098831qcucse
cju2y0z6g87p10878fpk5d3rq
cju2y26c588bo07993ksd8eoz
cju2y40d8ulqo0993q0adtgtb
cju2y5zas8m7f0801d34g5owq
cju2y8s56ymqr083541ggdsml
cju2yb31a8e8u0878wdashg7o
cju2ycp1u8g2r0799jslnp7cz
cju2yg5ht8i4p087800js8hp4
cju2yi9tz8vky0801yqip0xyl
cju2yljr0yzhw0988ecf271ly
cju2yo1j1v0qz09934o0e683p
cju2ysg748ru80878sp6j0gm0
cju2yv4imv6cz099314jveiib
cju2yw4s7z7p20988lmf2gdgd
cju2yyhsp933j0855hp32e012
cju2z1nxlzaj40835wj81s1iy
cju2z2x3nvd3c099350zgty7w
cju2z45kuzf6d0988nz2c819m
cju2z6ez69g4u0801qwt088lw
cju2z9vlp9j0w0801oag91sy9
cju2zblxw9848087853csbrx1
cju2zdhsczmn50988z64qwg2q
cju2zdvjn9h7r08553cp4eed5
cju2zgbj9zmrw0835nnlzxj4c
cju2zi4l09f5807991s8do2b2
cju2zjcvj9qma0801dk71hhi0
cju2zkpdl9h7t0799ix60teqg
cju2zm0axztpe0988r8s9twjr
cju2zo0fwzv580988qlijd2xa
cju2zp89k9q1g0855k1x0f1xa
cju2zpw4q9vzr0801p0lysjdl
cju2zr3c3vwb00993jn06bbaz
cju2zrojo9kcd0878ld2epejq
cju2zwg05a0oy0801yr73ig7g
cju2zxja9w1eh09933609ho9z
cju2zy1e49pqk0878t6ncqn12
cju300m3s04fg0988uzupuf7z
cju302fqq9spc0878rrygyzzz
cju303j5r062k098835zxfds5
cju30525w04r10835ygp257sb
cju306x7w05nb0835cunv799x
cju30ajhw09sx0988qyahx9s8
cju30bmab08bi0835mvlr6e0r
cju30df2j09dd08351ayx2t6w
cju30ftgja7170855xl9bkdm0
cju30gxjq0djk0988jytm49rs
cju30ia8da2bq0799klnehml2
cju30j1rgadut0801vuyrsnt8
cju30k2z40ds308353kdew70n
cju30lncba3ny0878jwnous8n
cju30mm25a53s0799qa5wiqe8
cju30nyxe0gfb0835p256yoju
cju30ov1oah920801mi8thuyg
cju30qbm1ad3x0855znuhpz9u
cju30u1hbakn808019g15nb8b
cju30xqmh0ni00835ix3batv1
cju30ywtc0oar0835bp2en7ec
cju310f6val1v0855xo8tc3gu
cju3128yi0rpu0988o4oo5n8n
cju31rb7vb6110801p9rhacuw
cju31t8xd17bk0835rnb893jk
cju31ugmfb3dz0855xtqshki6
cju31w6goazci0799n014ly1q
cju31y80qbawn0801twwm2l5s
cju320gyvbch60801v2amdi2g
cju323ypb1fbb0988gx5rzudb
cju324q101fhe08350wae9cif
cju326h4v1gxw08352px40p7r
cju3280wv1ir009882jze27tc
cju32a52lb9rc0799xi40qs00
cju32csyfblyh080170aa3x5p
cju32fhnhbds40799broyoptc
cju32gzs6xo8x0993r8tedbpb
cju32jcdabepz0878d0cznmfe
cju32l161bi1v07990vm376in
cju32phw2bv130801yj7bkouq
cju32pzh9bpw10855q4vaxfhe
cju33yemn2qb20988wfjxximx
cju33za6l2qy70988jhrlp2ev
cju34aozyyy830993bn16u32n
cju34c1xfyz920993itxkkfad
cju34ds2531520988qjpqt6e3
cju34eqjpcpm508788b3lhp97
cju34fojcctcf0799ebolbvkn
cju34i3qvcyog0855qiejxx5w
cju34m7h536wq0988xz7gx79v
cju34o6dbd2lo0855aqlcy1hs
cju34ouumcznz07996gg1xq7v
cju34repocy5208780gswillm
cju34sh43d8zm08019xbwhc0o
cju34uhepd3dd0799hs8782ad
cju34xspwzenf0993cyzajv9n
cju34ymm8d6700799uop0cw33
cju34zivp3fq80988opxbaqyn
cju3518w2d838079939fqztbc
cju3521y5d5mq0878t3ezsu4p
cju353d1eda8c07992afde611
cju35740hzm0g0993zl5ic246
cju357rxxdaz30878y2esjpjt
cju358pwtdby20878cg7nm0np
cju35a77vdj4n08556jj2lgmc
cju35atpxdjot0855q46aqrd0
cju35c4wzdhow0799h6eq4sgs
cju35eg0tdmjt085525sb4bua
cju35fxqyzt5p0993vusm54qz
cju35i2e63uxr0835h7zgkg9k
cju35k2fr3vc50988c85qkrwg
cju35ldepdtlm0801yv79y8vu
cju35mdz73x890835eynq1h9v
cju35oyvd3y850988km12hdz1
cju3tp94kfstl08181awh6z49
cju3tsh4lfsok0987w6x3a0v1
cju3ttznuftyf09875t11850w
cju3tvffffx5f0818t5ov22al
cju3tx1qyg0c907552fglumhc
cju3u1c8tfyqx08503iedc3mx
cju3u39fog1bo0871lxjrabks
cju3u4lxmg59o0755rz42b9en
cju3u815rg4ek0850vvhtcvcm
cju3ua8u0g9rg0801uayhdxhu
cju3uhb79gcgr0871orbrbi3x
cju3ul8dogf1z09872y2ecowp
cju3umoh1geet0817cmpef5am
cju3uwz6ogsp10801h2r3bj5l
cju3uz4o6gr9z0850lhxyxvsj
cju3v0fl3gwce0755qkjhzmd4
cju3v11mrgwwb0755u242ygye
cju3v3ac9gyz30755hfqwyp1i
cju3v56bwgy8v0871w14pz8fx
cju3v664kh0px0818y4y7wolf
cju3v72v5h1qz0818fggilwtq
cju3wstckialg0871xs0vevsj
cju3x2s11ibzi0817kk284k0j
cju3x4blzieu30850x10uuvbm
cju3x5u2tiihx0818914gzxy1
cju3x7xsaijq80818f0psavav
cju3x9lttikfb0818a0g104zn
cju3xeexgii1j0817zs68tb4g
cju3xga12iixg0817dijbvjxw
cju3xhpvvimda0987ygrpzni2
cju3xiic0ilzp0850lusrb42j
cju3xjqtpikx50817tppy6g84
cju3xl264ingx0850rcf0rshj
cju3xtufwiv9c0818djsc4cqd
cju3xuj20ivgp0818mij8bjrd
cju3xvoo2iqlc0817eku2r3wl
cju3xwpgviwlx0871rwm15q7v
cju3xzvnzj0hd0755xprz39nj
cju3y0pjrj1c30755nxekxccj
cju3y21quj0ir0818kgjagr15
cju3y54kwj3nr0801biidlb4e
cju3y79ofj3va0871uqfb1mzo
cju3y9difj6th0801kd1rqm3w
cju3ya7goj6at0818v2l5ay7f
cju3yb47cj1xq0817zfotbni4
cju3yht87j83m08507yk1u1fg
cju3ykamdj9u208503pygyuc8
cju40jl7skiuo0817p0smlgg8
cju40m0rjkpw80871z6n6yg1u
cju40poe4kt9s0755f9cnm3h5
cju40r6jrksyk0871wg98zgho
cju40sdwukv3k0755y99ug1k8
cju40taxlkrho0987smigg0x0
cju40u30gkuzc0871rq7t666d
cju40w3hbkwpn08015rbs3wko
cju40wto5kxwi0755it190f2k
cju410dnfl0960755y8lu8d79
cju412uwlkva50850d1ps1ww7
cju414lf2l1lt0801rl3hjllj
cju418rckl3ur08012psrx1r1
cju41kd7yl4nm0850gil5qqwh
cju41lojblbs307555jdci937
cju41nz76lcxu0755cya2qefx
cju41p90plcsx08018cnzpndc
cju41r6v2lcww0871ps8k8pf5
cju41s6nbleqy0755e2mslg0b
cju41z76wlgbz0801qdetlvby
cju422cm8lfxn0818ojicxejb
cju424hy5lckr085073fva1ok
cju426tomlhll0818fc0i7nvh
cju428k5fldt108177s6g6f45
cju42dwedlmk60871jbgu4ehi
cju42g865lorv07552ytz6xxa
cju42m60jlpcm08186kqppzqv
cju42nm68lpyo0818xvvqmupq
cju42py9mlqyd0818u3d1d7ga
cju42qet0lsq90871e50xbnuv
cju42romflni20817etb9a0fl
cju42tauqlo5p08171l3cuo4b
cju42u5bjlvi10801dc13sskp
cju42wamblrqn098798r2yyok
cju42xpi8lw4w0871ve317a1p
cju430pm2lz0y0755jkhcc3d1
cju439oazm2fu0871ma0vvrft
cju43b8daly4408170e5ev06g
cju43c92lm5cj0755lorsorfg
cju43eigtm6ev0801mv0m96t1
cju43gfosm63n08714rpih8pe
cju43h43am1dy08176gwfhmnt
cju43in5fm22c08175rxziqrk
cju43jcqim2cp08172dvjvyui
cju43kj2pm34f0850l28ahpni
cju43lcnum9y10755bjs7z87f
cju43mkj9m8wb0871qiadahub
cju43o6n7m9nk087191ijwqq9
cju45jpvfn6c809873pv1i34s
cju45lbgznahl08180xz1h7u6
cju45n0oxn5vu08500yfrt9jn
cju45ofd9ne1j0801ri8dup7t
cju45pm27n80u08174kyow1gj
cju45qbf3n9sa0987oonbkly9
cju45rj7ln8980850a7821fov
cju45t5ddnbio0987qtqzx762
cju45ty6zn9oz0850qy4qnck1
cju45v0pungu40871acnwtmu5
cju5b9oyda4yr0850g9viziyv
cju5bbtwsa8cl0987wgfsqpao
cju5bdwa3aatx0818b79i18zf
cju5bf6hxa6m50817rbwettgu
cju5bhv81abur0850ean02atv
cju5bj926aiec07559rshy4wa
cju5bmhdcafs909878qfzrqzi
cju5boicjagt20871b1fotkh4
cju5buy2bal250818ipl6fqwv
cju5bwhapakm90987c1v4z46a
cju5bycdkalkb09875f7bfrvx
cju5c5xc7algd0817pb1ej5yo
cju5c7oijaqmq09878qwgqv8n
cju5ca9hcatkc0801jzwe7tfx
cju5ccpvqash50850kb4bs22k
cju5cetivauok0987ok3e5bre
cju5chrxxawka0871qcj171yz
cju5cjh3xattc0817j2vbulzi
cju5cky5xb0ay0801oxet697t
cju5clr68b48r0755cmuvponm
cju5cu8qkb84x08186jwo8yin
cju5ddda9bkkt0850enzwatb1
cju5eftctcdbj08712gdp989f
cju5ekty5ckzf07550c9u3ckk
cju5enq1tcn1i0755hnkon787
cju5eq8c8ck690850vix98hv3
cju5es375cnzy0801nkq35ffs
cju5ew4h9cqaf0818rrczkmqh
cju5eyfe9cpk90987laa7tsl3
cju5f0dezct4q08183ydw11dx
cju5f26ebcuai0818xlwh6116
cju5f8hxdcxxn08188obby0ea
cju5fb86jd1jp0755b1ukbhq5
cju5fi0yxd3ei0801v7u0yudn
cju5fs6j6d8350801vglraq4u
cju5ft6mcd5q40987rhjgbrr6
cju5fu081d8gc0818l3yylujk
cju5fw37edaae0801vkwvocn7
cju5fydrud94708507vo6oy21
cju5g163vd6mt0817uccuga6u
cju5gucasds9d0801019axylx
cju5h57xedz5h0755mjpc8694
cju5hi52odyf90817prvcwg45
cju5hjxaae3i40850h5z2laf5
cju5hl8nee8a40755fm8qjj0o
cju5hqz50e7o90850e0prlpa0
cju5ht88gedbu0755xrcuddcx
cju5huurrecm70801y680y13m
cju5hwonqedw10801vsd3w6kk
cju5hyi9yegob0755ho3do8en
cju5i39mreass0817au8p22zy
cju5i5oh2efg60987ez6cpf72
cju5jx7jzf7c90871c2i9aiov
cju5jz5fff8c50871hbe6108f
cju5k3j3uf6de0817hszzfr7n
cju5k503sfa5f0871lx0rpu5y
cju5k7r0yf98c09878csbxb4d
cju5klveuff6w0871wbibgh3m
cju5knbbqfipk080128cggukq
cju5kre09fhka0850h7b1898j
cju5ktjwofed70817eg58ef7u
cju5nxkujgscq0817l9gss626
cju5nyu31gv8e0871zpk74a2n
cju5o1vu9gz8a0818eyy92bns
cju5o4pk9h0720755lgp9jq8m
cju5tenjojp1j0755ms4949h2
cju5tgbzhjllu08174ca41eus
cju5thdbrjp1108715xdfx356
cju5u4pywk81x0817vn9pe14z
cju5u6wf0kh1t0755bg1ssixv
cju5u8gz4kj5b07552e2wpkwp
cju5udcufki0s09874ll1dbr5
cju5ufn3skquf0818dhapnhba
cju5uget8krjy0818kvywd0zu
cju5uhrdwkmsu0817ervv91l8
cju5ukkg6kv7u08011x2b6zl5
cju5uxjnol2r509871qv2yeia
cju5uzmaol56l0817flxh4w9p
cju5v8pgplg6k0755rvi2t63h
cju5vbo6jldrt0871jf6f1700
cju5vcmrqla7i0817x4sp4pqw
cju5vgawslbe30987ndeepc1b
cju5vi4nxlc530817uoqm2m7a
cju5von04litr08718j8po40a
cju5vutu7ll8w0871dfp92n9p
cju5vwbr4lhqn0987a1pji0ux
cju5vxuc5loxw0818u8xgf45p
cju5vzjoslpj708186z2fusmz
cju5w7xn0lrkq0801f9k0htgx
cju5waeduln160817w0agirve
cju5wcc90lu020850mjrxppv6
cju5wi6bqlxy90755bu227nvb
cju5wj0faly5008187n6530af
cju5wkonqlrl409877y8zvnub
cju5wmvsdlx1j0871npgj8j4b
cju5woy82m07m08505dmjg7g1
cju5wphwwlu3m0987hh3ltg88
cju5wqonpm0e60801z88ewmy1
cju5wrapcm2290818jsh26ppb
cju5wrrs0m2af0818vmnajbtw
cju5wtdu4m0im0871mix0yvc0
cju5wuhm1lwm40987vugqn3vv
cju5x00l6m5j608503k78ptee
cju5x15djm7ae0755h8czf6nt
cju5x28nzm7t907558ocq4bt7
cju5x7iskmad90818frchyfwd
cju5xjn5mm78b09871spyqhhr
cju5xkwzxmf0z0818gk4xabdm
cju5xneamme2p0801qdf7fdwv
cju5xopi0md7q0871r1sjc1av
cju5xq3tdm9fn0987pbedxdg5
cju5y4hgqmk0i08180rjhbwvp
cju5y7buemcw80987p0r30g9f
cju5y84q3mdv50817eyp82xf3
cju5yclrymlgj0818k426ud6z
cju5yeqiwmkgl0801fzv2douc
cju5ygh1zmmdi0755uod5e17i
cju5yhgznmkzb0801cji2vi8j
cju5yimthmlv80850zhoc90c2
cju5yjq1pmlgc0801z0t24bly
cju5ymyd8mmdc0801ry3by1xr
cju6ur9l9v9jq0755paud9uka
cju6us80mv1b50871ebyq2wxa
cju6ut4l8va6y0755tyw3vfqq
cju6uy20suzbl0987rzuhz7z9
cju6uzxk0v83p0801rcwnexdu
cju6v1m1xv07w09870ah3njy1
cju6v3bb2v7xo085090blubyw
cju6v4szov55u0871qmqz3v8n
cju6v5ilsv8hk0850rb5sgh6o
cju6v6g6kvdw007552x6mb0po
cju6vgdmivcvb08018fra5lnv
cju6vifjlv55z0987un6y4zdo
cju6vqarjv7yo0987q4b1btk1
cju6vrs1ov8cr098788h8gs6j
cju6vta3kvazg0817qbeppjtm
cju6vucxvvlda0755j7msqnya
cju6vvb8svhed0801jjcquh5e
cju6vvxsev9y30987kespucdg
cju6w733bveoz0817e600tw72
cju6wi3akvn8r0801px8eligc
cju6wjm81vgsc0987enk9n3pr
cju6wll7wvo3y08502pagos8m
cju6wn57mvooj0850rp78hhy7
cju6wt9jvvn500871hjn3t3g0
cju6wuojavt740818b5qcv3iw
cju6x0yqbvxqt0755dhxislgb
cju6x35ervu2808015c7eoqe4
cju6x4t13vyw60755gtcf9ndu
cju6x97w4vwua0850x0997r0a
cju6xa0qmvzun0818xjukgncj
cju6xifswvwbo0987nibtdr50
cju6xlygpw7bs0818n691jsq4
cju6xmqd9w0250817l5kxfnsk
cju6ywm40wdbo0987pbftsvtg
cju6yxyt0wh080871sqpepu47
cju6yywx1whbb0871ksgfgf9f
cju6z1bzbwfq50817b2alatvr
cju6z2616wqbk07555bvnuyr1
cju6z600qwh4z081700qimgl9
cju6z7e4bwgdd0987ogkzq9kt
cju6z9a9kwsl007552s49rx6i
cju76erapykj30871x5eaxh4q
cju76l27oyrw907551ri2a7fl
cju76lsehyia10987u54vn8rb
cju76o55nymqd0871h31sph9w
cju77196iyshb0850ycbto50a
cju772304yw5t0818vbw8kkjf
cju773hsyyosz0817pk1e7sjq
cju774fmayxif0818u2g79usw
cju7787c5yy3l080159mwqsnj
cju77afzlz3kp07550x5nafzs
cju77b3wyz4160755qis4ljsb
cju77bvg0yv4r0987yh60xmjo
cju77g99iyxc00817zqi2ppor
cju77idwfz36d0871tzfzz51i
cju77j66ez52p08019xygi0co
cju77k828z46w0871r0avuoo9
cju77q10sz9ug0801449wu1nu
cju77re6fz5bb0817vp9redjg
cju77t0razbvm080106o56289
cju77u1sjz77b0817ft44r3fk
cju77vvcwzcm50850lzoykuva
cju783tmkzkqu081803g7q5vk
cju784jpdzeae0987q5ypq883
cju785htizjzo08017tvlhtg4
cju787jnjzjuj0871p94nck9g
cju7adqyj1jcx08712r1ro5gx
cju7ae7bq1f820987toc8si1d
cju7aez2x1jtj0871ztezs3oi
cju7afqon1ip40850ue2308b6
cju7agj961l2r0818z29iq8yn
cju7ahtkb1jr90801jck4kbds
cju7aifyo1p3n07552nxjx51f
cju7ajnbo1gvm098749rdouk0
cju7aklv31h4309871m29l4e7
cju7alcgr1lsr0871riqk84z7
cju7amjna1ly40871ugiokehb
cju7ap09p1kz10850ldccjebj
cju7apr0c1qqm0755s7msqot4
cju7aqkue1i2k09879uzcpt8r
cju7arvfe1ldu0850erdmphgj
cju7atnm31if40817pqclnjer
cju7avvi51iox0817ym55y6tt
cju7awzmu1ncs0871hziy65zx
cju7azuu31mia0801pf9ib5ed
cju7b10ce1mnm08011c5bwyr4
cju7b1ygu1msd0801hywhy0mc
cju7b2l561oas0871decgslaf
cju7b3f5h1sm40755i572jden
cju7b4mtw1n9n080186209f3d
cju7b5afm1nfw0801xqm8bf8q
cju7b9vcs1luz0987ta60j1dy
cju7bb3ss1uoo0755pmhyco7t
cju7bc95p1mdm0817yqj5jc6j
cju7bd1qu1mx409877xjxibox
cju7bduyq1rjf08719giru9ho
cju7bf1lp1shi081835vs84lc
cju7bfx651qr80801cs7epotb
cju7bgnvb1sf808717qa799ir
cju7bmi1v1pnj0987pa52jjok
cju7cl8zm1xcu0817ado0jpas
cju7cp6dw244p0818gncdol4m
cju7cq6su27qv075574dir0r3
cju7crgxa28550755wbsgqkel
cju7csvlb22fr0850lvm45n3x
cju7ctvqn25dy08186g442m1r
cju7cue9b232j0801qdzk1ykj
cju7cufm7298k0755j09uf3of
cju7d1tvt25bu08019dvw3uff
cju7d2q1k27nf08715zshsckt
cju7d3oc82cho0755dajlwldz
cju7d4jk723eu0817bqz2n39m
cju7d5m0p23kn09871rk7pu3v
cju7d6ux323ze0987xos3srkx
cju7d7aut2a2p0818z4uxc6cd
cju7d7tly27h408016fyp5nr7
cju7d8m3b2e210755l8fj1yph
cju7d9seq29zd0871nzl2uu5m
cju7da88w2eod0755wejzynvt
cju7db7lp2f400755tntd1ohf
cju7dbppn28nx085097654msi
cju7dda8w2br20818zhsuz8s7
cju7ddtz729960801uazp1knc
cju7deifq2fzn0755lc8idyh8
cju7dff9529h208503w60lbil
cju7dglf226g50987ohbthl19
cju7dhpsc2dnn0818025m6857
cju7dizi82h2i0755doucgnt3
cju7dlk532dsh0871zvr6qz0r
cju7dmlgf2ebw0871ieqas5fh
cju7dn24o296i09871qfxb8s2
cju7do8c72dbo0801vxfzxdc4
cju7ehljc2or70871261br8ai
cju7ejm2l2ncl0801wq6y84nw
cju7ekbo32pft0871fv7kzwb9
cju7el6xv2k520817qxx9wdr5
cju7emdni2py40871ivhxjtut
cju7eotqi2qea0871y8yc7tqh
cju7epwj82koz098713apjnzo
cju7er4kc2opa0801anuxc0eb
cju7es23b2vcp0755gpbm9s7v
cju7et17a2vjk0755e743npl1
cju7etr3y2p4t0801cdzjj8ab
cju7eueum2oqn0850rodmx8zo
cju7ev2b12owa08500bpfpwyw
cju7evxt12m730987rxivne3x
cju7ey10f2rvf0871bwbi9x82
cju7ez7r22qbc08015xfoz2wb
cju7ezs7g2mxm098787atbran
cju7f0ec32txj08184asb8w5f
cju7f4sc62xqj075597xpmuoy
cju7f5ghb2r5s0801chwkxxh9
cju7f6cqy2ur20818t1saazbm
cju7f900s2o0k08175gl1giid
cju7f9umg2olj0987fj5y285w
cju7fazv92ywx0755xov2erga
cju7fbndk2sl608015ravktum
cju7fcgbe2z3p07550vaflqdb
cju7fd6yt2p740987wkr8exo1
cju7fen322ou10817ziqkob4k
cju7ff97z2ow40817u2r83my5
cju7flevb2wii08188otgs9p2
cju7fmvpk2q170987v6i3ola8
cju7fnfv02tt90801djnix9m8
cju7fob3x301u0755x985pmmq
cju7fpfzq2wyf0818xxd1oziv
cju7fq7mm2pw508176uk5ugtx
cju7frtqu2xa20818wq8r9fzf
cju7g7ba42z310987bqzbi2bq
cju83h9ysjwe808716nt35oah
cju83ipu3jwpx0801z5pvguf8
cju83k8fyjsxr0817d6nxs6r4
cju83kxitjv340987z09m0ezy
cju83mki1jv5w0817kubxm31r
cju83nwu1jxte0987h1krpfmv
cju83qd0yjyht0817ktkfl268
cju83rcnzkbsj0755x5anfrcg
cju83syhdk6gs0801rf1rekdl
cju83u9ftk3ni0987qnhlcinv
cju83vvmik9wa08710yeh7cuk
cju83wwn1k55e0850kw6i2d81
cju83yddek68q0850d2x7zfkm
cju8402x1kcy70801t6kz6bdi
cju8418jhkf7d0818ga2v0xq0
cju8432cmkgq90871cxe4iptl
cju843yjskhq30818qre4rwm2
cju846ec0kj7z08012o10klrb
cju847pxykriq0755268ktrk2
cju849c23kgnk0817cgv2hw1e
cju84aoa3ktwn0755pfl4gfwd
cju84dsvaklpx098750hp83x4
cju84ffdzkrjn08183jh1fxmb
cju84gpefknwm098714oq8q61
cju84hibuktj80871u519o71q
cju84ih17kp5l09876bkooocl
cju84jdl9kv0i0871eog9b3i9
cju84kplnl1y30755ropua1b0
cju8567gdlcbq0801dwwyo2jt
cju857ad0l88m0817qx4cwxnf
cju858eswlepn0871pzvdrhj1
cju85a8h8llwm07559wxg4t5w
cju85bf1algsq0871y9gtlq97
cju85c2d4ln1b0755zz1z3onx
cju85citjlnfm0755i4rk5tqj
cju85dx63lic408017f0l0400
cju85fc11ljr40818edpb0inh
cju85hguellg50818kwu3s8d1
cju85ia1slh220987y7c20sm2
cju85je7vlht70817c9jcjwi4
cju85l4yjlops0801fvmnwptf
cju85mpuglq8k0818d2it6hzb
cju885ikhnmkn09878s2lqtuh
cju886ryxnsl50801r93jai7q
cju887ftknop008177nnjt46y
cju888fr7nveu0818r9uwtiit
cju88aq6vo1ij0755c2ey7z7n
cju88cddensj00987788yotmg
cju88evxanv9r08176zkeovec
cju88fpm4o0tl0871w1i6a4ds
cju88gx09o2vk0818610zody3
cju88itqbny720987hxizbj5y
cju88k75inzyb0850ccv5x3vk
cju88l66no10s0850rsda7ej1
cju88msmoo3470817m441j4sg
cju88nroho44508500129f1nh
cju88oh0po9gq0801nge4tgr1
cju88q6h6obpd0871ckmiabbo
cju88rl5eo94l0850kf5wtrm1
cju88t4fvokxf07558ymyh281
cju88trl3ogi208716qvti51b
cju88v2f9oi8w0871hx9auh01
cju88vx2uoocy075531lc63n3
cju88y1mwoln50871emyfny1g
cju88z8bson4h0871nnd7fdxo
cju890guyoiti098753yg6cdu
cju8914beokbf0850isxpocrk
cju892fesoq2g0801n0e0jyia
cju893jmdompz0817xn3g1w4h
cju89y9h0puti0818i5yw29e6
cju89z6pqpqfx0817mfv8ixjc
cju8a1jtvpt9m081712iwkca7
cju8a2itsq4dv0755ntlovpxe
cju8a3nhbpwnb0850d37fo2na
cju8a56vxpy780850r45yu4wk
cju8a84g0q76m0818hwiggkod
cju8abobpqbir08189u01huru
cju8adb60qbiu080188mxpf8d
cju8aeei7q8k308173n9y4klv
cju8aj01yqeqm0850lhdz3xdw
cju8alhigqn2h0801zksudldd
cju8amfdtqi4x09871tygrgqe
cju8ando2qqdo0818ck7i1be1
cju8apjewqrk00801k5d71gky
cju8aqq8uqmoq0987hphto9gg
cju8arof2qpf20850ifr1bnqj
cju8ashhnquqr0801rwduzt7d
cju8at3s1qqqx0850hcq8nmnq
cju8auylgqx0z0871u4o4db7o
cju8aw9n1qyg10801jkjlmors
cju8axq24r4an0755yhv9d4ly
cju8ayeq7r1fb0818z1junacy
cju8azmhcr66e0755t61atz72
cju8b0jr0r2oi0801jiquetd5
cju8b1v3br45u087189kku66u
cju8b2rmgr52s0801p54eyflx
cju8b3ka8r64u0801fh18hk7l
cju8b4ja9r2s808509d45ma86
cju8b542nr81x0871uxnkm9ih
cju8b5p40r2c60987ofa0mu03
cju8b6rp0r5st0850184f79xt
cju8b7aqtr4a00987coba14b7
cju8b8yair65w09878pyqtr96
cju8bafgqrf4x0818twisk3ea
cju8bbznkrf5g0871jncffynk
cju8bff9nrfi10850fmfzbf8v
cju8bgdmqrksy0801tozdmraa
cju8bh8surexp0987o5pzklk1
cju8bi8q7rlmn0871abc5ch8k
cju8bj2ssrmlm0871gc2ug2rs
cju8bk8oirjhw0817hgkua2w8
cju8bljw9rqk20801kr54akrl
cju8bm24yrrdp081829mbo8ic
cju8bn7m2rmm70817hgxpb1uq
cju8bop5jrsid08716i24fqda
cju8bpctzrqkr0850zeldv9kt
cju8bqxxurs6i0850mu7mtef9
cju8brv16rx7f0818uf5n89pv
cju8bssulrrcy0987h1vq5060
cju8buos5rz9b08715lfr0f4f
cju8bw697rwg308177tg8huas
cju8bysfgrzkl081786jwac09
cju8bzzy2s66m08016z6mouqt
cju8c1a0ws7o208181c6lbsom
cju8c2rqzs5t80850d0zky5dy
cju8c3xs7sauj0801ieyzezr5
cju8c5223s8j80850b4kealt4
cju8c5mxls96t0850wvkvsity
cju8c5zcbsdfz0801o5t6jag1
cju8c6hnxsdvr0801wn0vrsa6
cju8c82iosagu0817l74s4m5g
cju8c9akjsdjj0850s67uzlxq
cju8ca4geseia0850i2ru11hw
cju8cattbsivm0818p446wgel
cju8cbsyssiqj0871gr4jedjp
cju8cdeazsm8h0801jxifmzur
cju7efffp2ivf0817etg3jehl"""

test_split = """cju0s690hkp960855tjuaqvv0
cju0sr5ghl0nd08789uzf1raf
cju0sxqiclckk08551ycbwhno
cju0t4oil7vzk099370nun5h9
cju0tl3uz8blh0993wxvn7ly3
cju0u2g7pmnux0801vkk47ivj
cju0u82z3cuma0835wlxrnrjv
cju0ue769mxii08019zqgdbxn
cju0vtox5ain6099360pu62rp
cju13cgqmnhwn0988yrainhcp
cju13fwthn9mq0835gacxgy01
cju13hp5rnbjx0835bf0jowgx
cju14g8o4xui30878gkgbrvqj
cju14hjh2ob2o0835ouz3r5aa
cju14pxbaoksp0835qzorx6g6
cju15czxqp3lv0835jvhgzurz
cju15jr8jz8sb0855ukmkswkz
cju15l5ubz9yh0855b3ivdpse
cju15mhjczc8z0801kit5c6di
cju15ptjtppz40988odsm9azx
cju15wdt3zla10801odjiw7sy
cju160wshltz10993i1gmqxbe
cju16ach3m1da0993r1dq3sn2
cju16b6ynq8e40988m8vx0xnj
cju16d65tzw9d0799ouslsw25
cju16fpvhzypl0799p9phnlx6
cju16jgnyzp970878melv7r25
cju16whaj0e7n0855q7b6cjkm
cju32qr9tbvsj08013pkpjenq
cju32srle1xfq083575i3fl75
cju32upim1z7u0988l883nqp6
cju32zhbnc1oy0801iyv1ix6p
cju330ofbc2l30801th5g3hw6
cju33231uy4gi0993qc7b1jch
cju334jzo261t0835yqudnfs1
cju336l68y7if0993wf092166
cju3381d8bz3h07991xtl7ra0
cju33belnbyhm0878yxl42233
cju33eqwbcch208012jikwdky
cju33jon3ygbj0993pu22a4k6
cju33mirdc8mj0799k33wzoes
cju33o12x2jm50988944mxq0v
cju33qpdvc9g0087825jhf3s9
cju33w4sdcivk0855x879zht7
cju33x0f22peh0988g0ln7w5v
cju8ceacrsqkr0755hdz145es
cju8cgi2kspp308011nxdtjp6
cju8chdlqsu620755azjty1tj
cju8chxndsre008015uisl4si
cju8cj10qsrau0871o2dr6ai1
cju8clorgsuwn08714toqb7v6
cju8cwy02t9eq08185qn12c02
cju8czvnztbf40871b4m7t78w
cju8d2q30tfhs0801n7lx77xl
cju8d4jgatgpj0871q2ophhkm
cju8dic9mtppa0987swn23wbc
cju8djdqztu6408506pzhlo18
cju8dk7eztzup08182yxko5zh
cju8dm2cau2km0818jsv9eeq2
cju8dn0c3u2v50801k8rvq02f
cju8doa16u5gh0818w1ywda3q
cju8dpa89u6l80818dj6lldh9
cju8dqkrqu83i0818ev74qpxq
cjyzjzssvd8pq0838f4nolj5l
cjyzk8qieoboa0848ogj51wwm
cjyzkmjy8evns070165gf9dmq
cjyzkpsbjdsjq07211dfi4sru
cjyzl833ndne80838pzuq6ila
cjyzlw7f9faqr070129au64sq
cjyztzaqtrv430848l8xgcerw
cjyzu3reghjya0794w7pwoi50
cjyzu9th0qt4r0a46pyl4zik0
cjyzufihqquiw0a46jatrbwln
cjyzuio1qgh040763k56deohv
cjyzul1qggwwj07216mhiv5sy
cjyzurzvohqnr0794es1itzek
cjz14qsk2wci60794un9ozwmw
ck2395w2mb4vu07480otsu6tw
ck2bxiswtxuw80838qkisqjwz
ck2bxknhjvs1x0794iogrq49k
ck2bxlujamu330725szlc2jdu
ck2bxpfgxu2mk0748gsh7xelu
ck2bxqz3evvg20794iiyv5v2m
ck2bxskgxxzfv08386xkqtqdy
ck2bxw18mmz1k0725litqq2mc
ck2da7fwcjfis07218r1rvm95
cju85nr8elly209872w9n5m0s
cju85omszllp30850b6rm9mi3
cju85plp7lmkw0850rx42jdpf
cju85qefyln6v0850szeb9byi
cju85rkbnlo1c08503uxcpax1
cju87kbcen2av0987usezo8kn
cju87li0zn3yb0817kbwgjiz8
cju87mrypnb1e0818scv1mxxg
cju87nkyrnb970801q84m47yt
cju87ox0kncom0801b98hqnd2
cju87q6yoneim0871dl4phvkd
cju87r56lnkyp0755hz30leew
cju87tyddnnad0755bj0wxahe
cju87vqa0ndwg0850onjdz7ol
cju87xn2snfmv0987sc3d9xnq
cju87z6o6nh73085045bzsx6o
cju87zv8lni0o0850hbbecbq6
cju8828oxnool0801qno9luhr
cju884985nlmx0817vzpax3y4
cju7dp3dw2k4n0755zhe003ad
cju7dqcwi2dz00850gcmr2ert
cju7druhp2gp308715i6km7be
cju7dsrtb2f8i085064kwugfk
cju7dtb1e2j0t0818deq51ib3
cju7dubap2g0w0801fgl42mg9
cju7dvl5m2n4t0755hlnnjjet
cju7dwe282dc309876rco45ts
cju7dxffn2eam0817qxosfwch
cju7dymur2od30755eg8yv2ht
cju7dz5yy2i7z0801ausi7rna
cju7ea4om2l910801bohqjccy
cju7ebe962hr409872ovibahw
cju7ecl9i2i060987xawjp4l0
cju7eea9b2m0z0801ynqv1fqu"""

In [11]:
#@title loading dataset paths
train_names = train_split.splitlines()
val_names = test_split.splitlines()
data_dir = "/kaggle/working/Kvasir-SEG"
train_images = [f"{data_dir}/images/{f}.jpg" for f in train_names]
train_masks = [f"{data_dir}/masks/{f}.jpg" for f in train_names]
val_images = [f"{data_dir}/images/{f}.jpg" for f in val_names]
val_masks = [f"{data_dir}/masks/{f}.jpg" for f in val_names]

In [12]:
train_df = pd.DataFrame({"image_path": train_images, "mask_path": train_masks})
val_df = pd.DataFrame({"image_path": val_images, "mask_path": val_masks})

In [13]:
img_aug = ImageDataGenerator(**wandb.config["train_data_augmentation_config"])
msk_aug = ImageDataGenerator(**wandb.config["train_data_augmentation_config"])

image_train_gen = img_aug.flow_from_dataframe(dataframe=train_df,
                                            x_col="image_path",
                                            batch_size=wandb.config["batch_size"],
                                            seed=wandb.config["seed"],
                                            class_mode=None,
                                            target_size=(wandb.config["img_height"],wandb.config["img_width"]),
                                            color_mode='rgb')

mask_train_gen = msk_aug.flow_from_dataframe(dataframe=train_df,
                                            x_col="mask_path",
                                            batch_size=wandb.config["batch_size"],
                                            seed=wandb.config["seed"],
                                            class_mode=None,
                                            target_size=(wandb.config["img_height"],wandb.config["img_width"]),
                                            color_mode='grayscale')

Found 880 validated image filenames.
Found 880 validated image filenames.


In [14]:
val_img_aug = ImageDataGenerator(rescale=1./255.)
val_msk_aug = ImageDataGenerator(rescale=1./255.)

image_val_gen = val_img_aug.flow_from_dataframe(dataframe=val_df,
                                            x_col="image_path",
                                            batch_size=wandb.config["batch_size"],
                                            seed=wandb.config["seed"],
                                            class_mode=None,
                                            target_size=(wandb.config["img_height"],wandb.config["img_width"]),
                                            color_mode='rgb')

mask_val_gen = val_msk_aug.flow_from_dataframe(dataframe=val_df,
                                            x_col="mask_path",
                                            batch_size=wandb.config["batch_size"],
                                            seed=wandb.config["seed"],
                                            class_mode=None,
                                            target_size=(wandb.config["img_height"],wandb.config["img_width"]),
                                            color_mode='grayscale')

Found 120 validated image filenames.
Found 120 validated image filenames.


In [15]:
def data_iterator(image_gen,mask_gen):
    for img, mask in zip(image_gen,mask_gen):
        yield img, mask
val_gen = data_iterator(image_val_gen, mask_val_gen)
train_gen = data_iterator(image_train_gen, mask_train_gen)

In [16]:
#@title dice metrics
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

In [17]:
#@title model building blocks

def channel_attention_module(x, ratio=wandb.config["cbam_ratio"]):
    batch, _, _, channel = x.shape

    ## Shared layers
    l1 = Dense(channel//ratio, activation="relu", use_bias=False)
    l2 = Dense(channel, use_bias=False)

    ## Global Average Pooling
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)

    ## Global Max Pooling
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)

    ## Add both the features and pass through sigmoid
    feats = x1 + x2
    feats = Activation("sigmoid")(feats)
    feats = Multiply()([x, feats])

    return feats

def spatial_attention_module(x):
    ## Average Pooling
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)

    ## Max Pooling
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)

    ## Concatenat both the features
    feats = Concatenate()([x1, x2])
    ## Conv layer
    feats = Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(feats)
    feats = Multiply()([x, feats])

    return feats

def cbam(x):
    x = channel_attention_module(x)
    x = spatial_attention_module(x)
    return x


def residual_block(x, num_filters):
    x_init = x

    x = Conv2D(num_filters, 3, padding="same", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    s = Conv2D(num_filters, 1, padding="same", kernel_initializer='he_normal')(x_init)
    s = BatchNormalization()(x)
    #s = SqueezeAndExcite2D(num_filters)(s)
    s = cbam(s)

    x = Activation("relu")(x + s)

    return x

def strided_conv_block(x, num_filters):
    x = Conv2D(num_filters, 3, strides=2, padding="same", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x


def encoder_block(x, num_filters):
    x1 = residual_block(x, num_filters)
    x2 = strided_conv_block(x1, num_filters)
    with options({"layout_optimizer": False}):
        x2 = SpatialDropout2D(0.3)(x2)
    x3 = residual_block(x2, num_filters)
    p = MaxPool2D((2, 2))(x3)

    return x1, x3, p


In [18]:
def build_colonsegnet(input_shape):
    """ Input """
    inputs = Input(input_shape)
    
    c1 = Conv2D(64,  2, strides=2, padding='same')(inputs)
    c2 = Conv2D(256,  2, strides=2, padding='same')(c1)
    mp = MaxPool2D((2,2))(c2)

    """ Encoder """
    s11, s12, p1 = encoder_block(inputs, 64)
    s21, s22, p2 = encoder_block(p1, 256)

    """ Decoder 1 """
    x = Conv2DTranspose(128, 4, strides=4, padding="same")(s22 + mp)
    x = Concatenate()([x, s12 + c1])
    with options({"layout_optimizer": False}):
        x = SpatialDropout2D(0.25)(x)
    x = residual_block(x, 128)
    r1 = x

    x = Conv2DTranspose(128, 4, strides=2, padding="same")(s21 + c2)
    x = Concatenate()([x, r1])
    with options({"layout_optimizer": False}):
        x = SpatialDropout2D(0.25)(x)
    x = residual_block(x, 128)

    """ Decoder 2 """
    x = Conv2DTranspose(64, 4, strides=2, padding="same")(x)
    x = Concatenate()([x, s11])
    with options({"layout_optimizer": False}):
        x = SpatialDropout2D(0.25)(x)
    x = residual_block(x, 64)
    r2 = x

    x = Conv2DTranspose(64, 4, strides=2, padding="same")(s12 + c1)
    x = Concatenate()([x, r2])
    x = residual_block(x, 32)

    """ Output """
    output = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    """ Model """
    model = Model(inputs, output, name="ColonSegNet")

    return model

In [19]:
input_shape = (wandb.config["img_height"],wandb.config["img_width"], wandb.config["img_channels"])
model = build_colonsegnet(input_shape)
model.summary()

Model: "ColonSegNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d_2[0][0]']               
 alization)                     )                                                       

In [20]:
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """Cosine decay schedule with warm up period.

    Cosine annealing learning rate as described in:
      Loshchilov and Hutter, SGDR: Stochastic Gradient Descent with Warm Restarts.
      ICLR 2017. https://arxiv.org/abs/1608.03983
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.

    Arguments:
        global_step {int} -- global step.
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
    Returns:
      a float representing learning rate.

    Raises:
      ValueError: if warmup_learning_rate is larger than learning_rate_base,
        or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(tf.keras.callbacks.Callback):
    """Cosine decay with warmup learning rate scheduler
    """

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """Constructor for cosine decay with warmup learning rate scheduler.

    Arguments:
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        global_step_init {int} -- initial global step, e.g. from previous checkpoint.
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
        verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))

In [21]:
!mkdir /kaggle/working/models
loss = sm.losses.bce_dice_loss
metrics = [
    dice_coef,
    sm.metrics.IOUScore(threshold=0.5, name="IoU"),
    sm.metrics.Precision(threshold=0.5, name="Precision"),
    sm.metrics.Recall(threshold=0.5, name="Recall"),
    sm.metrics.FScore(threshold=0.5, name="F1Score"),
    ]

logger = WandbCallback(save_model=False,
                       generator=val_gen,
                       validation_steps=int(wandb.config["dataset_splits"]["val"]/wandb.config["batch_size"]),
                       predictions=3,
                       input_type="auto",
                       output_type="segmentation_mask",
                       log_evaluation=True,
                       infer_missing_processors=True,
                       log_evaluation_frequency=25,
                       )
model_checkpointer = WandbModelCheckpoint("/kaggle/working/models/", save_freq='epoch')


warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=wandb.config["learning_rate"],
                                            total_steps=110*wandb.config["epochs"],
                                            warmup_learning_rate=0.00001,
                                            warmup_steps=int((50*110)/8),
                                            hold_base_rate_steps=110*5,
                                            verbose=0)

model.compile(optimizer=tf.keras.optimizers.Adam(wandb.config["learning_rate"]),
              loss=loss,
              metrics=metrics,
              )

In [ ]:
hist = model.fit(train_gen,
                 epochs=wandb.config["epochs"],
                 validation_data=val_gen,
                 batch_size=wandb.config["batch_size"],
                 steps_per_epoch=int(wandb.config["dataset_splits"]["train"]/wandb.config["batch_size"]),
                 validation_steps=int(wandb.config["dataset_splits"]["val"]/wandb.config["batch_size"]),
                 callbacks=[warm_up_lr, logger, model_checkpointer])
wandb.run.finish()

Epoch 1/200
4/4 [==============================] - 14s 2s/step


wandb: Adding directory to artifact (/kaggle/working/models)... Done. 0.2s


110/110 [==============================] - 242s 2s/step - loss: 1.5151 - dice_coef: 0.2372 - IoU: 0.1794 - Precision: 0.2002 - Recall: 0.6913 - F1Score: 0.2982 - val_loss: 1.6510 - val_dice_coef: 0.2817 - val_IoU: 0.2200 - val_Precision: 0.2310 - val_Recall: 0.8420 - val_F1Score: 0.3563
Epoch 2/200
1/1 [==============================] - 0s 32ms/step- loss: 1.2541 - dice_coef: 0.2432 - IoU: 0.2016 - Precision: 0.2768 - Recall: 0.4351 - F1Score: 0.32


wandb: Adding directory to artifact (/kaggle/working/models)... Done. 0.2s


110/110 [==============================] - 70s 637ms/step - loss: 1.2541 - dice_coef: 0.2432 - IoU: 0.2016 - Precision: 0.2768 - Recall: 0.4351 - F1Score: 0.3281 - val_loss: 1.2313 - val_dice_coef: 0.2666 - val_IoU: 0.1855 - val_Precision: 0.3083 - val_Recall: 0.3319 - val_F1Score: 0.3108
Epoch 3/200
1/1 [==============================] - 0s 30ms/step- loss: 1.1651 - dice_coef: 0.2508 - IoU: 0.1971 - Precision: 0.3039 - Recall: 0.3658 - F1Score: 0.32


...

110/110 [==============================] - 71s 643ms/step - loss: 0.4242 - dice_coef: 0.7311 - IoU: 0.6260 - Precision: 0.7375 - Recall: 0.8074 - F1Score: 0.7654 - val_loss: 0.3401 - val_dice_coef: 0.8150 - val_IoU: 0.7243 - val_Precision: 0.8716 - val_Recall: 0.8156 - val_F1Score: 0.8363
Epoch 198/200
1/1 [==============================] - 0s 34ms/step- loss: 0.4157 - dice_coef: 0.7364 - IoU: 0.6330 - Precision: 0.7560 - Recall: 0.7984 - F1Score: 0.77


wandb: Adding directory to artifact (/kaggle/working/models)... Done. 0.2s


110/110 [==============================] - 71s 645ms/step - loss: 0.4157 - dice_coef: 0.7364 - IoU: 0.6330 - Precision: 0.7560 - Recall: 0.7984 - F1Score: 0.7709 - val_loss: 0.2911 - val_dice_coef: 0.8333 - val_IoU: 0.7568 - val_Precision: 0.8668 - val_Recall: 0.8584 - val_F1Score: 0.8591
Epoch 199/200
1/1 [==============================] - 0s 31ms/step- loss: 0.4177 - dice_coef: 0.7342 - IoU: 0.6297 - Precision: 0.7489 - Recall: 0.7976 - F1Score: 0.76


wandb: Adding directory to artifact (/kaggle/working/models)... Done. 0.3s


110/110 [==============================] - 71s 647ms/step - loss: 0.4177 - dice_coef: 0.7342 - IoU: 0.6297 - Precision: 0.7489 - Recall: 0.7976 - F1Score: 0.7680 - val_loss: 0.3332 - val_dice_coef: 0.8234 - val_IoU: 0.7410 - val_Precision: 0.8738 - val_Recall: 0.8338 - val_F1Score: 0.8466
Epoch 200/200
103/110 [===========================>..] - ETA: 3s - loss: 0.4209 - dice_coef: 0.7332 - IoU: 0.6285 - Precision: 0.7383 - Recall: 0.8101 - F1Score: 0.7678